# Project 3 - Part 2 Extract from TMDB (Core)
<b> Your Stakeholder Wants More Data!
    
- After investigating the preview of your data from Part 1, your stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).
    - This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
- Your stakeholder identified The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!
- Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".
- Note: this process can take a long time and may need to run overnight.


In [1]:
# imports
import json, os, time
import tmdbsimple as tmdb
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [2]:
# load API 
with open('/Users/j29ma/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
# API key
tmdb.API_KEY =  login['api-key']

In [4]:
# folder for API call data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [5]:
# make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [6]:
# movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/giBJ0ezYNEobFBfB8H4JNTjmll.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 63.096,
 'poster_path': '/aOIuZAjPaRIE6CMzbazvcHuHXDc.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film 

## Query Movies by ID
### IMDB ID

In [7]:
info['imdb_id']

'tt0133093'

### Budget

In [8]:
info['budget']

63000000

#### Revenue

In [9]:
info['revenue']

463517383

### Certification

In [11]:
# example from package README
# source = https://github.com/celiao/tmdbsimpleresponse = movie.releases()
releases = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


# <b> Specifications - Financial Data
    
- Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

- As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

- Each year should be saved as a separate .csv.gz file

Hint: Use the two custom functions from the lessons (Intro to TMDB API, and Efficient TMDB API Calls). Be sure to define these functions prior to calling them in your code!

- One function will add the certification (MPGG Rating) to movie.info
- The other function will help you append/extend a JSON file with Python
    
<b> Confirm Your API Function works.

In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

Hint: Ideally you can organize the code segments from the previous lesson to create an outer and inner loop, but if you get stuck, you can complete 1 year at a time.

- Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.

## Certification (MPGG Rating) Function

In [12]:
# function to add certification to movie.info dict
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    # set movie id
    movie = tmdb.Movies(movie_id)

    # save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # only get releases of the movie in the US
    for c in releases['countries']:
        
        # if country abbreviation == US
        if c['iso_3166_1'] == 'US':
            
            # save certification key into info dict
            info['certification'] = c['certification']
            
    return info

In [24]:
def information(movie):
    print(f'Budget: ', movie['budget'])
    print(f'Revenue: ', movie['revenue'])
    print(f'Certification: ', movie['certification'])

### 1st Test

In [30]:
movie_id1 = 'tt0848228'
info1 = get_movie_with_rating(movie_id1)

In [31]:
information(info1)

Budget:  220000000
Revenue:  1518815515
Certification:  PG-13


### 2nd Test

In [34]:
movie_id2 = 'tt0332280'
info2 = get_movie_with_rating(movie_id2)

In [35]:
information(info2)

Budget:  29000000
Revenue:  115603229
Certification:  PG-13


## Extend JSON file Function

In [ ]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

# <b> Exploratory Data Analysis
    
- Load in your csv.gz's of results for each year extracted.
    1. Concatenate the data into 1 dataframe for the remainder of the analysis.
- Once you have your data from the API, they would like you to perform some light EDA to show:
    1. How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
        1. Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
    2. How many movies are there in each of the certification categories (G/PG/PG-13/R)?
    3. What is the average revenue per certification category?
    4. What is the average budget per certification category?

<b> Deliverables
    
After you have joined the tmdb results into 1 dataframe in the EDA Notebook,

- Save a final merged .csv.gz of all of the tmdb api data
- The file name should be "tmdb_results_combined.csv.gz"
- Make sure this is pushed to your github repository along with all of your code
    - One code file for API calls
    - One code file for EDA